In [1]:
from hft_backtest_engine.data_loader import DataLoader
from hft_backtest_engine.feature_store import FeatureStore
from hft_backtest_engine.execution import ExecutionEngine
from hft_backtest_engine.backtest_engine import BacktestEngine
from hft_backtest_engine.strategy_base import Strategy, StrategyConfig
from hft_backtest_engine.run_backtest import estimate_vpin_bucket_volume
import pandas as pd

# =====================
# USER PARAMETERS
# =====================
DATA_ROOT = r"C:/Users/elvin/OneDrive - SKKU/Quant/백테스팅 데이터"
SYMBOL = "DOGEUSDT"
DAY = "2024-03-01"

INITIAL_CAPITAL = 100.0

# 🔧 핵심 튜닝 포인트
TARGET_VPIN_BUCKETS = 720      # 👈 여기 바꾸면서 실험
VPIN_GATE_THRESHOLD = 0.7

# =====================
# RUN
# =====================
def main():
    loader = DataLoader(DATA_ROOT)

    # 1) VPIN bucket 자동 산정
    bucket_vol = estimate_vpin_bucket_volume(
        loader,
        SYMBOL,
        DAY,
        TARGET_VPIN_BUCKETS,
    )
    print(f"[VPIN] bucket_volume = {bucket_vol:,.2f}")

    # 2) FeatureStore
    feature_store = FeatureStore(
        symbol=SYMBOL,
        vpin_bucket_volume=bucket_vol,
        vpin_min_buckets=20,
        vpin_gate_threshold=VPIN_GATE_THRESHOLD,
    )

    # 3) Strategy
    strategy = Strategy(
        symbol=SYMBOL,
        feature_store=feature_store,
        config=StrategyConfig(),
        initial_capital=INITIAL_CAPITAL,
    )

    # 4) Execution``
    execution = ExecutionEngine(
        latency_ms=5,
        slippage_bp=1.0,
        fill_price_mode="limit",
    )

    # 5) Engine
    engine = BacktestEngine(
        loader=loader,
        strategy=strategy,
        execution=execution,
        feature_store=feature_store,
        initial_capital=INITIAL_CAPITAL,
        verbose=True,
        debug_ticks=0,
    )

    engine.run_day(SYMBOL, DAY)

    fills_df = pd.DataFrame(engine.fills)
    closed_df = pd.DataFrame(engine.closed_trades)

    print(f"[RESULT] fills={len(fills_df)} closed={len(closed_df)}")

    return fills_df, closed_df


if __name__ == "__main__":
    fills_df, closed_df = main()

[VPIN] bucket_volume = 26,450,259.62
[DONE] DOGEUSDT 2024-03-01 ticks=747838 fills=0 closed=0
[RESULT] fills=0 closed=0


In [4]:
closed_df

""


In [5]:
from hft_backtest_engine.metrics import compute_metrics, plot_cum_pnl
import pandas as pd

INITIAL_CAPITAL = 100.0

# =========================
# 1) fills / closed 확인
# =========================
if closed_df is None or closed_df.empty:
    print("❌ closed trades 없음 → 성과지표 계산 불가")
else:
    required_cols = {"exit_ts", "net_pnl"}
    if not required_cols.issubset(closed_df.columns):
        raise ValueError(f"closed_df columns missing: {required_cols - set(closed_df.columns)}")

    # =========================
    # 2) equity curve 생성 (단리)
    # =========================
    equity_curve = (
        closed_df
        .sort_values("exit_ts")
        .assign(
            equity=lambda x: INITIAL_CAPITAL + x["net_pnl"].cumsum(),
            cash=lambda x: INITIAL_CAPITAL + x["net_pnl"].cumsum(),
            n_positions=0,
        )
        .loc[:, ["exit_ts", "equity", "cash", "n_positions"]]
        .rename(columns={"exit_ts": "ts"})
        .reset_index(drop=True)
    )

    display(equity_curve.head())

    # =========================
    # 3) 성과지표 계산
    # =========================
    metrics = compute_metrics(
        equity_curve=equity_curve,
        fills=fills_df if fills_df is not None and not fills_df.empty else None,
        initial_capital=INITIAL_CAPITAL,
    )

    print("\n=== METRICS ===")
    for k, v in metrics.items():
        print(f"{k:>25}: {v}")

    # =========================
    # 4) 단리 누적 PnL 그래프
    # =========================
    plot_cum_pnl(
        equity_curve,
        initial_capital=INITIAL_CAPITAL,
        title=f"{SYMBOL} {DAY} Cumulative PnL (Simple)",
    )


❌ closed trades 없음 → 성과지표 계산 불가
